In [0]:
!git clone https://github.com/pruvi007/Age-Estimation-Models

Cloning into 'Age-Estimation-Models'...
remote: Enumerating objects: 2225, done.
remote: Total 2225 (delta 0), reused 0 (delta 0), pack-reused 2225
Receiving objects: 100% (2225/2225), 88.94 MiB | 45.77 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [0]:
import os
import cv2
import numpy as np
from keras.utils import to_categorical

Using TensorFlow backend.


In [0]:
def hotVector(n,classes):
    vector = [ 0 for i in range(classes) ]
    vector[n-1] = 1
    vector = np.array(vector)
    return vector

In [0]:
# get the images from directory
dir = "Age-Estimation-Models/FGNET/FGNET/images"
images = []
for file in os.listdir(dir):
    img = cv2.imread(dir+"/"+file,0)
    images.append(img)
images = np.array(images)
print(np.shape(images[0]))
# resize the images for uniformity
min_x, min_y = 1000,1000
for img in images:
    min_x, min_y = min(min_x,np.shape(img)[0]), min(min_y,np.shape(img)[1])
print(min_x,min_y)
for i in range(len(images)):
    width = min_x
    height = min_y
    dim = (width, height)
    resized = cv2.resize(images[i], dim, interpolation = cv2.INTER_AREA)
    images[i] = resized

ages = []
for file in os.listdir(dir):
    s = file.split('.')
    a = []
    n = len(s[0])
    for i in range(n-1,-1,-1):
        if s[0][i]=='A':
            break
        a.append(s[0][i])
    if a[0] in ['a','b','c','d','e']:
        a.pop(0)
    age = 0
    for i in range(len(a)-1,-1,-1):
        age = age*10 + int(a[i])
    
    # print(file)
    # print(age)
    ages.append(age)
ages = np.array(ages)
# ages = to_categorical(ages)
print(np.shape(ages))

(447, 369)
359 300
(1002,)


In [0]:

perc = 50
train_x = []
train_y = []
for i in range((len(images)*perc)//100):
    train_x.append( images[i].reshape(359,300,1) )
    train_y.append( ages[i] )
train_x = np.array(train_x)
train_y = np.array(train_y)

test_x,test_y = [],[]
for i in range((len(images)*perc)//100,len(images)):
    test_x.append( images[i].reshape(359,300,1) )
    test_y.append( ages[i] )
test_x = np.array(test_x)
test_y = np.array(test_y)

print(np.shape(train_x))
print(np.shape(train_y))
print(np.shape(test_x))
print(np.shape(test_y))


(501, 359, 300, 1)
(501,)
(501, 359, 300, 1)
(501,)


In [0]:
import tensorflow as tf 
import matplotlib.pyplot as plt 
from keras.models import Sequential 
from keras.layers import Dense,Conv2D,Dropout,Flatten,MaxPooling2D

input_shape = (359,300,1)
model = Sequential()

model.add(Conv2D(64,kernel_size=(5,5),input_shape=input_shape,activation='relu'))
model.add(MaxPooling2D(pool_size=(3,3)))

model.add(Conv2D(32,kernel_size=(3,3)))
model.add(MaxPooling2D(pool_size=(3,3)))

model.add(Conv2D(16,kernel_size=(3,3)))


model.add(Conv2D(16,kernel_size=(3,3)))





model.add(Flatten())  # flatten to form the FC layer
model.add(Dense(70,activation=tf.nn.softmax))
print(model.summary())


model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['mae'])
model.fit(x=train_x,y=train_y,epochs=10,batch_size=50)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 355, 296, 64)      1664      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 118, 98, 64)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 116, 96, 32)       18464     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 38, 32, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 36, 30, 16)        4624      
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 34, 28, 16)        2320      
_________________________________________________________________
flatten_2 (Flatten)          (None, 15232)            

In [0]:
scores = model.evaluate(test_x, test_y, verbose=0)
print("%s: %.5f%%" % (model.metrics_names[1], scores[1]))

mae: 15.80037%
